<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/SPI1_regression/SPI1_genomewide_regression_labelsgen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download hg38 fasta file
![[ -e hg38.fa ]] || wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz -O hg38.fa.gz
![[ -e hg38.fa ]] || gunzip hg38.fa.gz

#install pyfaidx and prepare an index for hg38.fa
!pip install pyfaidx
from pyfaidx import Fasta
Fasta("hg38.fa") #this will build the index if it does not exist

#download hg38 chromsizes file
![[ -f hg38.chrom.sizes ]] || wget http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes -O hg38.chrom.sizes

#download and install bedtools
!apt-get install bedtools
  
#download the replicate-combined fc-over-control bigwig track for hg38
![[ -e fc.bigWig ]] || wget https://www.encodeproject.org/files/ENCFF289XSX/@@download/ENCFF289XSX.bigWig -O fc.bigWig
#download the optimal idr thresholded peaks for hg38
![[ -e optimal_idr.bed.gz ]] || wget https://www.encodeproject.org/files/ENCFF071ZMW/@@download/ENCFF071ZMW.bed.gz -O optimal_idr.bed.gz
#download the replicate-combined "peaks and background" set, from which we will construct the ambiguous peaks
![[ -e peaks_and_bg.bed.gz ]] || wget https://www.encodeproject.org/files/ENCFF492DUT/@@download/ENCFF492DUT.bed.gz -O peaks_and_bg.bed.gz
  
#treat the top 150K in the "peaks and bg" set that don't overlap the idr optimal set as the ambiguous set
!bedtools intersect -v -a <(zcat peaks_and_bg.bed.gz | head -150000) -b optimal_idr.bed.gz -wa | gzip -c > ambiguous.bed.gz

--2019-08-31 01:25:17--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  81.9MB/s    in 19s     

2019-08-31 01:25:36 (50.3 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]

  Created wheel for pyfaidx: filename=pyfaidx-0.5.5.2-cp36-none-any.whl size=24641 sha256=5273733ca94502a2befee0e6290d939a43686080a092c0812f77abee195b1c1a
  Stored in directory: /root/.cache/pip/wheels/54/a2/b4/e242e58d23b2808e191b214067880faa46cd2341f363886e0b
Successfully built pyfaidx
--2019-08-31 01:27:54--  http://hgdownload.cse.ucsc.edu/goldenPath/hg38/bigZips/hg38.chrom.sizes
Resolving hgdownload.cse.ucsc.edu (hgdownload.cse.ucsc.edu)... 128.11

In [3]:
with open("tasks.tsv",'w') as f: 
    f.write("mytask\toptimal_idr.bed.gz\tfc.bigWig\tambiguous.bed.gz\n")
f.close() 
!cat tasks.tsv

mytask	optimal_idr.bed.gz	fc.bigWig	ambiguous.bed.gz


In [5]:
!git clone https://github.com/kundajelab/seqdataloader.git
!pip install seqdataloader/

Cloning into 'seqdataloader'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 878 (delta 69), reused 83 (delta 33), pack-reused 743
Receiving objects: 100% (878/878), 3.89 MiB | 8.24 MiB/s, done.
Resolving deltas: 100% (545/545), done.
Processing ./seqdataloader
     |████████████████████████████████| 194kB 4.9MB/s 
     |████████████████████████████████| 12.5MB 26.2MB/s 
     |████████████████████████████████| 71kB 23.0MB/s 
     |████████████████████████████████| 3.2MB 35.3MB/s 
     |████████████████████████████████| 51kB 21.2MB/s 
  Created wheel for seqdataloader: filename=seqdataloader-0.126-cp36-none-any.whl size=25586 sha256=b1ded79bc313bae9b818d62a5a1b4377a79ce35556747c3e9a680ead34828ba2
  Stored in directory: /tmp/pip-ephem-wheel-cache-58yiwhtn/wheels/c2/db/13/112d41662f69fb8c7986c218293570cc1550fc21eed966e31b
  Created wheel for deeptools: filename=deepTools-3.3.0-cp36-

In [18]:
from seqdataloader.labelgen import genomewide_labels


train_set_params={
    'task_list':"tasks.tsv",
    'outf':"regression_labels.tsv.gz",
    'output_type':'gzip',
    'chrom_sizes':'hg38.chrom.sizes',
    'chroms_to_keep':("chr1 chr2 chr3 chr4 chr5 chr6 chr7 chr8 chr9"
                      +"chr10 chr11 chr12 chr13 chr14 chr15 chr16"
                      +"chr17 chr18 chr19 chr20 chr21 chr22 chrX chrY").split(" "),
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'chrom_threads':10,
    'task_threads':1,
    'allow_ambiguous':True,
    'labeling_approach':'peak_percent_overlap_with_bin_regression',
    'store_positives_only': True
    }
genomewide_labels(train_set_params)


creating dictionary of bed files and bigwig files for each task:
mytask
creating chromosome thread pool
pre-allocated df for chrom:chr11with dimensions:(2701713, 4)
in get_labels_one_task
starting chromosome:chr11 for task:mytask
pre-allocated df for chrom:chr8with dimensions:(2902753, 4)
in get_labels_one_task
starting chromosome:chr8 for task:mytask
pre-allocated df for chrom:chrXwith dimensions:(3120798, 4)
in get_labels_one_task
starting chromosome:chrX for task:mytask
pre-allocated df for chrom:chr7with dimensions:(3186900, 4)
in get_labels_one_task
starting chromosome:chr7 for task:mytask
pre-allocated df for chrom:chr6with dimensions:(3416100, 4)
in get_labels_one_task
starting chromosome:chr6 for task:mytask
pre-allocated df for chrom:chr5with dimensions:(3630746, 4)
in get_labels_one_task
starting chromosome:chr5 for task:mytask
pre-allocated df for chrom:chr4with dimensions:(3804272, 4)
in get_labels_one_task
starting chromosome:chr4 for task:mytask
pre-allocated df for chrom

In [19]:
!zcat regression_labels.tsv.gz | wc -l

280593


In [20]:
!zcat universal_negatives.regression_labels.tsv.gz | wc -l

51980898


In [24]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!mkdir /content/gdrive/"My Drive"/colab_notebook_data/regression_experiments/SPI1_GM12878

In [0]:
!cp regression_labels.tsv.gz /content/gdrive/"My Drive"/colab_notebook_data/regression_experiments/SPI1_GM12878
!cp universal_negatives.regression_labels.tsv.gz /content/gdrive/"My Drive"/colab_notebook_data/regression_experiments/SPI1_GM12878

In [0]:
!echo "code to generate labels is at https://github.com/AvantiShri/colab_notebooks/blob/8250de10703d7962aa595a535595d3dccb57c841/SPI1_regression/SPI1_genomewide_regression_labelsgen.ipynb" > /content/gdrive/"My Drive"/colab_notebook_data/regression_experiments/SPI1_GM12878/README.txt


In [34]:
!md5sum *.tsv.gz

eb61b0defda6bc825dcb985d5fbb7da0  regression_labels.tsv.gz
6b1c0d2b1945c62f414fe3b59bae1153  universal_negatives.regression_labels.tsv.gz
